# Knapsack

## Model Formulation

### Sets and Indices

$i \in I$: Index and set of items.

### Parameters

$v_{i} \in \mathbb{R}^+$: Value of item $i \in I$.

$w_{i} \in \mathbb{R}^+$: Weight of item $i \in I$.

$K \in \mathbb{R}^+$: Capacity of the knapsack

### Decision Variables

$x_{i} \in \{0, 1 \}$: This variable is equal to 1 if we take item $i \in I$; and 0 otherwise.

### Objective Function

- **Total value**. We want to maximize the total value of the items selected to go into the knapsack. This is the sum of the values of the selected items.

\begin{equation}
\max Z= \displaystyle \sum_{i=0}^{n-1} v_ix_i
\tag{0}
\end{equation}

### Constraints

- **Capacity**. The total weight of the selected items $i \in I$ must not exceed the capacity $K$ of the knapsack:

\begin{equation}
\displaystyle \sum_{i=0}^{n-1} w_ix_i \leq K \text{ where } x_i \in \{0,1\} \ \forall i \in \{ 0, \ldots, n-1 \}
\tag{1}
\end{equation}

# Python implementation

## Import the libraries

The following code imports the required libraries.

In [6]:
from collections import namedtuple
import pandas as pd

ModuleNotFoundError: No module named 'ortools'

## Tuple definition

In [ ]:
Item = namedtuple("Item", ['index', 'value', 'weight'])

## Create the data

The code below creates the data for the problem.  

### Read the file

In [ ]:
url = 'https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/data/ks_30_0'
df = pd.read_csv(url, sep=" ", header=None)
df.head()

### Read the data

The data includes the following:

**weights**: A vector containing the weights of the items.

**values**: A vector containing the values of the items.

**capacities**: A vector with just one entry, the capacity of the knapsack.

In [ ]:
item_count = int(df.at[0,0])
capacity = int(df.at[0,1])
capacities = []
capacities.append(capacity)

print("Number of items =", item_count)
print("Capacity of the knapsack =", capacity)

values = []
weights = [[]]

for i in range(1, item_count+1):
    values.append(int(df.at[i,0]))
    weights[0].append(int(df.at[i,1]))

print("Values:", values)
print("Weights:", weights)

## Declare the solver

The following code declares the knapsack solver, a specialized solver for knapsack problems.

In [ ]:
solver = knapsack_solver.KnapsackSolver(
        knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        "KnapsackExample",
    )

The option KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER tells the solver to use the branch and bound algorithm to solve the problem.

The knapsack solver works over the integers, so the data in the program can only contain integers. If your problem contains non-integer values, you can first convert them to integers by multiplying the data by a sufficiently large integer.

## Call the solver

The following code calls the solver.

In [ ]:
solver.init(values, weights, capacities)
computed_value = solver.solve()
print("Total value =", computed_value)

The program first initializes the solver, and then calls it by computed_value = solver.Solve(). The total value of the optimal solution is computed_value.

In [ ]:
taken = [0]*item_count
packed_items = []
packed_values = []
packed_weights = []
total_value = 0
total_weight = 0

for i in range(item_count):
    if solver.best_solution_contains(i):
        taken[i] = 1
        packed_items.append(i)
        packed_values.append(values[i])
        packed_weights.append(weights[0][i])
        total_value += values[i]
        total_weight += weights[0][i]

remaining_capacity = capacity-total_weight
print("Total value:", total_value)
print("Total weight:", total_weight)
print("Packed items:", packed_items)
print("Packed values:", packed_values)
print("Packed weights:", packed_weights)
print("Remaining capacity=", remaining_capacity)

## Prints the solution

Prints the solution in the specified output format

In [ ]:
output_data = str(computed_value) + ' ' + str(1) + '\n'
output_data += ' '.join(map(str, taken))

print(output_data)

## Visualize the solution

In [ ]:
import altair as alt

# dictionary of lists 
dict = {'item': packed_items, 'value': packed_values, 'weight': packed_weights} 
    
df_items = pd.DataFrame(dict)

bars = alt.Chart(df_items).mark_bar().encode(
    y='sum(value)',
    color='item:N'
).properties(
    width=100
)

pie = alt.Chart(df_items).mark_arc().encode(
    theta="weight",
    color="item:N"
).properties(
    width=300
)

alt.hconcat(
    pie ,bars).configure_axis(
    grid=False,
).configure_view(
    strokeWidth=0
)

# Shell script execution

## Upload data

In [ ]:
%%shell

wget -nc -P ./data https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/data/data.zip
cd data
unzip data.zip
rm data.zip
ls

## Execute solver

In [ ]:
%%shell

wget -nc https://raw.githubusercontent.com/jacubero/Optimization/main/knapsack/or_branch_and_bound/solver.py

mkdir -p or_branch_and_bound

for file in $(ls ./data/*)
do
  filename="$(basename "$file")"
  python3 solver.py $file > ./or_branch_and_bound/$filename.orb
done